Adal burada öneri motorunu deneyecek.

In [1]:
import pandas as pd #pandas'ı pd olarak çektik.

In [2]:
# 1. adım: Verileri okuduk (Bir üst klasöre çıkıp data içine giriyoruz)
df_details = pd.read_csv('../data/processed/clean_ordersdetails.csv')
df_products = pd.read_csv('../data/processed/clean_products.csv')

In [3]:
# 2. adım: Sadece ihtiyacım olan sütunları filtreledim (bellek tasarrufu)
df_details = df_details[['OrderID', 'ProductID']]
df_products = df_products[['ProductID', 'ProductName']]

In [4]:
# 3. adım: Tabloları ProductID üzerinden birleştir
df_merged = df_details.merge(df_products, on='ProductID', how='left')

In [5]:
# Test: İlk 5 satıra bakalım
print(df_merged.head())

   OrderID  ProductID                    ProductName
0    10248         11                 Queso Cabrales
1    10248         42  Singaporean Hokkien Fried Mee
2    10248         72         Mozzarella di Giovanni
3    10249         14                           Tofu
4    10249         51          Manjimup Dried Apples


In [6]:
#? Kesişim matrisi: Aynı sipariş numarasındaki (OrderID) tüm ürünleri gruplayacağız (groupby) ve 
# ürün isimlerini (ProductName) sütunlara yayacağız (unstack). 
# Eğer o siparişte o ürün yoksa, oluşan boşluklara 0 basacağız.

# 1. Her siparişte hangi üründen bulunduğunu gruplayıp tabloyu yana genişletiyoruz
basket = (df_merged.groupby(['OrderID', 'ProductName'])
          .size()
          .unstack()
          .fillna(0)) # Boş kalan yerlere (alınmayan ürünlere) 0 yazıyoruz

# 2. Sayıları 1 ve 0'a çeviren dönüştürücü fonksiyonumuz
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

# 3. Bu kuralı tüm tabloya uyguluyoruz
basket_sets = basket.map(encode_units)

# Tablonun yeni matris haline bakalım (test, doğru mu yazdım?)
print(basket_sets.head())

ProductName  Alice Mutton  Aniseed Syrup  Boston Crab Meat  Camembert Pierrot  \
OrderID                                                                         
10248                   0              0                 0                  0   
10249                   0              0                 0                  0   
10250                   0              0                 0                  0   
10251                   0              0                 0                  0   
10252                   0              0                 0                  1   

ProductName  Carnarvon Tigers  Chai  Chang  Chartreuse verte  \
OrderID                                                        
10248                       0     0      0                 0   
10249                       0     0      0                 0   
10250                       0     0      0                 0   
10251                       0     0      0                 0   
10252                       0     0      0      

In [7]:
# basket_sets tablosunu Apriori algoritmasına yedireceğiz. v1

from mlxtend.frequent_patterns import apriori, association_rules

# 1. En az %1 oranında birlikte geçen ürün kümelerini bul, hepsini alırsak ram gg
# use_colnames=True demezsek ürün isimleri yerine 0,1,2 gibi anlamsız sayılar verir.
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

# 2. Kuralları oluştur (3 Silahşörler: Support, Confidence, Lift)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# 3. Bizim için en değerli olan "Lift" (Kaldıraç/Artış) oranına göre büyükten küçüğe sırala
rules = rules.sort_values('lift', ascending=False)

# İlk 10 kurala bakalım, test
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))

Empty DataFrame
Columns: [antecedents, consequents, support, confidence, lift]
Index: []


c:\Projects\ecommerce-sales-analytics\venv\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:175: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [ ]:
# basket_sets tablosunu Apriori algoritmasına yedireceğiz. v2

from mlxtend.frequent_patterns import apriori, association_rules

# 1. 1 ve 0'ları bool'a çevirdik
basket_sets_bool = basket_sets.astype(bool)

# 2. BEST PRACTICE: max_len=2 ekliyoruz. Sadece ikili eşleşmelere bakacak, hızı 100 kat artacak.
# Çıtayı da 0.002 (binde iki) yapalım.
frequent_itemsets = apriori(basket_sets_bool, min_support=0.002, use_colnames=True, max_len=2)

# 3. Kuralları çıkar (Lift değeri 1'den büyük olanlar)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# 4. En yüksek Lift değerine göre sırala
rules = rules.sort_values('lift', ascending=False)

# İlk 10 kurala bakalım!
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))

                        antecedents                     consequents   support  \
858                    (Röd Kaviar)               (Mishi Kobe Niku)  0.002410   
859               (Mishi Kobe Niku)                    (Röd Kaviar)  0.002410   
326                   (Ipoh Coffee)                     (Chocolade)  0.002410   
327                     (Chocolade)                   (Ipoh Coffee)  0.002410   
309    (Northwoods Cranberry Sauce)  (Chef Anton's Cajun Seasoning)  0.003614   
308  (Chef Anton's Cajun Seasoning)    (Northwoods Cranberry Sauce)  0.003614   
533                          (Tofu)  (Grandma's Boysenberry Spread)  0.003614   
532  (Grandma's Boysenberry Spread)                          (Tofu)  0.003614   
890     (Queso Manchego La Pastora)    (Northwoods Cranberry Sauce)  0.002410   
891    (Northwoods Cranberry Sauce)     (Queso Manchego La Pastora)  0.002410   

     confidence       lift  
858    0.142857  23.714286  
859    0.400000  23.714286  
326    0.071429   9.8